In [1]:
!wget https://raw.githubusercontent.com/fluentpython/example-code/master/19-dyn-attr-prop/oscon/data/osconfeed.json

--2022-10-30 09:22:39--  https://raw.githubusercontent.com/fluentpython/example-code/master/19-dyn-attr-prop/oscon/data/osconfeed.json
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 744228 (727K) [text/plain]
Saving to: ‘osconfeed.json’

osconfeed.json      100%[===================>] 726.79K  2.47MB/s    in 0.3s    

2022-10-30 09:22:40 (2.47 MB/s) - ‘osconfeed.json’ saved [744228/744228]



In [7]:
class A:
    var = 10
    pass

a = A()
a.__getattribute__('var'), a.var

(10, 10)

In [15]:
import json
with open('osconfeed.json') as fp:
    feed = json.load(fp)

In [22]:
d = dict(feed) 
hasattr(d, 'a')

False

In [27]:
class Item:
    def __init__(self, val) -> None:
        self.val = val

    @property
    def val(self):
        return self.__val 

    @val.setter
    def val(self, val):
        self.__val = val